# Using LLama Factory finetune on SageMaker 
# 2. 使用vLLM进行本地推理

## 安装依赖包

In [1]:
!pip install vllm==0.5.5 bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.6/134.6 MB 83.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.2/797.2 MB 38.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 MB 89.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.8/20.8 MB 87.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 49.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 86.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 84.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 MB 90.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 4.1 MB/s eta 0:00:00eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### 从s3下载模型文件到本地

In [2]:
import boto3
import pprint
from tqdm import tqdm
import sagemaker
sagemaker_session =  sagemaker.session.Session() #sagemaker.session.Session()
region = sagemaker_session.boto_region_name
role = sagemaker.get_execution_role()
default_bucket = sagemaker_session.default_bucket()

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [9]:
!aws s3 sync s3://{default_bucket}/llama3-8b-qlora/ ./local_model

download: s3://sagemaker-us-east-1-434444145045/llama3-8b-qlora/finetuned_model/adapter_config.json to local_model/finetuned_model/adapter_config.json
download: s3://sagemaker-us-east-1-434444145045/llama3-8b-qlora/finetuned_model/README.md to local_model/finetuned_model/README.md
download: s3://sagemaker-us-east-1-434444145045/llama3-8b-qlora/finetuned_model/all_results.json to local_model/finetuned_model/all_results.json
download: s3://sagemaker-us-east-1-434444145045/llama3-8b-qlora/finetuned_model/checkpoint-500/README.md to local_model/finetuned_model/checkpoint-500/README.md
download: s3://sagemaker-us-east-1-434444145045/llama3-8b-qlora/finetuned_model/checkpoint-500/rng_state.pth to local_model/finetuned_model/checkpoint-500/rng_state.pth
download: s3://sagemaker-us-east-1-434444145045/llama3-8b-qlora/finetuned_model/checkpoint-500/adapter_config.json to local_model/finetuned_model/checkpoint-500/adapter_config.json
download: s3://sagemaker-us-east-1-434444145045/llama3-8b-qlor

## 加载模型tokenizer

In [10]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [11]:
model_id = 'TechxGenus/Meta-Llama-3-8B-Instruct-AWQ'
tokenizer = AutoTokenizer.from_pretrained(model_id)

## 加载sample数据，用于对比

In [12]:
from datasets import load_dataset
from random import randrange
# dataset_name = "zxbsmk/webnovel_cn"
dataset_name = "hfl/ruozhiba_gpt4"
# Load dataset from the hub
train_dataset = load_dataset(dataset_name, split="train",revision='41d2c61beb86c8d4c61916cc656c39d018c40ce5')

In [13]:

print(f"Training size: {len(train_dataset)}")
print("\nTraining sample:\n")
num_samples = 200
print(train_dataset[randrange(num_samples)])

Training size: 4898

Training sample:

{'input': '', 'instruction': '既然高考是看排名的，那大家为什么不都考0分呢? 这样人人都有大学上了', 'output': '这个提议存在一个根本性的逻辑错误。如果所有考生都考0分，高考的基本功能就失去了，因为高考的排名机制是为了区分不同学生的学术能力，以便分配教育资源。如果大家都考0分，无法区分谁有资格上大学，也没法合理分配教育资源。此外，大学的学习难度和要求也需要一定的基础知识，如果入学没有任何学术依据，学生在大学的学习中很难跟上课程。因此，0分的想法实际上是对高考制度和教育体系严重误解，不可能实现人人都上大学的目标。'}


In [14]:
sql_lora_path = './local_model/finetuned_model'

## 使用本地的vLLM部署

In [15]:
from vllm.lora.request import LoRARequest
from vllm import LLM,SamplingParams
from transformers import AutoModelForCausalLM, AutoTokenizer


In [16]:
model_id = 'TechxGenus/Meta-Llama-3-8B-Instruct-AWQ'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [17]:
llm = LLM(model=model_id,max_model_len=4096,enable_lora=True)

config.json:   0%|          | 0.00/885 [00:00<?, ?B/s]

INFO 11-02 11:44:10 awq_marlin.py:89] The model is convertible to awq_marlin during runtime. Using awq_marlin kernel.
WARNING 11-02 11:44:10 config.py:1451] awq_marlin quantization is not tested with LoRA yet.
INFO 11-02 11:44:10 llm_engine.py:184] Initializing an LLM engine (v0.5.5) with config: model='TechxGenus/Meta-Llama-3-8B-Instruct-AWQ', speculative_config=None, tokenizer='TechxGenus/Meta-Llama-3-8B-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None, col

generation_config.json:   0%|          | 0.00/152 [00:00<?, ?B/s]

INFO 11-02 11:44:11 model_runner.py:879] Starting to load model TechxGenus/Meta-Llama-3-8B-Instruct-AWQ...
INFO 11-02 11:44:11 weight_utils.py:236] Using model weights format ['*.safetensors']


model-00001-of-00002.safetensors:   0%|          | 0.00/4.68G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.05G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/63.5k [00:00<?, ?B/s]

Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 11-02 11:51:42 model_runner.py:890] Loading model weights took 5.3492 GB
INFO 11-02 11:51:52 gpu_executor.py:121] # GPU blocks: 6466, # CPU blocks: 2048
INFO 11-02 11:51:54 model_runner.py:1181] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-02 11:51:54 model_runner.py:1185] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-02 11:52:20 model_runner.py:1300] Graph capturing finished in 26 secs.


In [18]:
#测试第一个消息
messages = [
    {"role": "system", "content":"请始终用中文回答"},
     {"role": "user", "content": "你是谁？你是干嘛的"},
]

#测试第二个消息
# messages = [
#     {"role": "system", "content":"请始终用中文回答"},
#      {"role": "user", "content": "睡觉时被女鬼压床我该怎么办？"},
# ]


inputs = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

### 使用原始模型进行推理

In [19]:
sampling_params = SamplingParams(temperature=0.1, top_p=0.95,max_tokens=512)

outputs = llm.generate(inputs, sampling_params)
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt:\n{prompt!r}")
    print(f"Response:\n{generated_text!r}")


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.80s/it, est. speed input: 16.64 toks/s, output: 71.53 toks/s]

Prompt:
'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n请始终用中文回答<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n你是谁？你是干嘛的<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
Response:
'我是 LLaMA，一个由 Meta AI 开发的语言模型。我是一个人工智能语言模型，旨在与用户交谈，回答问题，提供信息和帮助。我的能力包括：\n\n* 回答问题：我可以回答各种问题，包括历史、科学、技术、文化、娱乐等领域。\n* 提供信息：我可以提供相关信息，帮助用户了解某个主题或问题。\n* 对话：我可以与用户进行对话，回答问题，提供建议和帮助。\n\n我是一个机器人，旨在帮助用户获取信息，解决问题和提高语言能力。'


### 加载Lora进行推理

In [20]:
sql_lora_path = './local_model/finetuned_model'

In [21]:
outputs = llm.generate(inputs, sampling_params,lora_request=LoRARequest("adapter", 1, sql_lora_path))

# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt:\n{prompt!r}")
    print(f"Response:\n{generated_text!r}")

/tmp/ipykernel_29027/343688066.py:1: DeprecationWarning: The 'lora_local_path' attribute is deprecated and will be removed in a future version. Please use 'lora_path' instead.
  outputs = llm.generate(inputs, sampling_params,lora_request=LoRARequest("adapter", 1, sql_lora_path))
Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.41it/s, est. speed input: 42.50 toks/s, output: 56.67 toks/s]

Prompt:
'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n请始终用中文回答<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n你是谁？你是干嘛的<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'
Response:
'您好，我是 Riverbot，一个由 Riverbot 开发的人工智能助手。我可以回答各种问题、提供信息和解决方案，帮助用户解决问题和满足需求。'
